# Homework 3: MCMC
### Jake Pitkin
**CS 6190: Probabilistic Modeling - Spring 2018**<br>
**April 7 2018**

$1.$ Write a function to perform Gibbs sampling of a binary label image $x$ with the Ising model prior and iid Gaussian likelihood, given a noisy image $y$. This function should take $\alpha$, $\beta$, and $\sigma$ parameters, and generate a random binary image (labels in the set $\{-1,1\}$ according to the posterior Gibbs distribution for $x|y.$ The energy should look like this:

$$U(x) = -\alpha \sum_{i} x_i - \beta \sum_{\langle i,j\rangle} x_ix_j + \frac{1}{2\sigma^2} \sum_{i} (x_i - y_i)^2$$

Note this is assuming that the $x_i$ labels are also the mean pixel values in the Gaussian. The $\alpha$ parameter controls the proportion of labels that are $-1$ versus $+1$. Negative values of $\alpha$ will favor more $-1$ pixels, and positive values of $\alpha$ will favor more $+1$ pixels.



In [103]:
import math
import random
from PIL import Image

#IMAGE_WIDTH = None
#IMAGE_HEIGHT = None
#Y = None
#X = None

def read_image(path):
    img = Image.open(path, 'r')
    pixels = img.load()
    width = img.size[0]
    height = img.size[1]
    Y = [[(pixels[x,y]/255) * 20 - 10 for x in range(IMAGE_WIDTH)] for y in range(IMAGE_HEIGHT)]
    #X = [[0 for x in range(IMAGE_WIDTH)] for y in range(IMAGE_HEIGHT)]
    return Y, width, height
    
def get_neighbours(X, row, col, width, height):
    # left, right, up, and down initialized to wrap values
    neighbours = [X[row][width-1], X[row][0], X[height-1][col], X[0][col]]
    # check if we don't have to wrap and replace
    if col != 0: # left
        neighbours[0] = X[row][col-1]
    if col != width-1: #right
        neighbours[1] = X[row][col+1]
    if row != 0: #up
        neighbours[2] = X[row-1][col]
    if row != height-1:
        neighbours[3] = X[row+1][col]
    return neighbours

def random_init_matrix(width, height):
    return [[random.choice([-1, 1]) for x in range(width)] for y in range(height)]

def ising_prior_norm(alpha, beta, X, row, col, width, height):
    prob = alpha * X[row][col]
    for n in get_neighbours(X, row, col, width, height):
        prob += beta * X[row][col] * n
    prob = math.exp(prob)
    Z = 
    return math.exp(prob)

def energy(alpha, beta, sigma, row, col):
    neighbours = get_neighbours(row, col)
    energy = -1 * alpha * X[row][col]
    for n in neighbours:
        energy -= beta * X[row][col] * n
    energy += (1/(2*math.pow(sigma, 2))) * math.pow(X[row][col]-Y[row][col], 2)
    return energy

def energy_norm(alpha, beta, sigma, row, col, x):
    neighbours = get_neighbours(row, col)
    energy = -1 * alpha * x
    for n in neighbours:
        energy -= beta * x * n
    energy += (1/(2*math.pow(sigma, 2))) * math.pow(x-Y[row][col], 2)
    return energy

def cond_dist_single_pixel(alpha, beta, sigma, row, col):
    cond_dist = math.exp(-1 * energy(alpha, beta, sigma, row, col))
    print(cond_dist)
    normalize = math.exp(-1 * energy_norm(alpha, beta, sigma, row, col, 1))
    print(normalize)
    normalize += math.exp(-1 * energy_norm(alpha, beta, sigma, row, col, -1))
    print(normalize)
    #return cond_dist / normalize
    

**(a)** Run your code with just the Ising prior term (no posterior). Do this a few times and generate several random binary images. Try different $\alpha$ and $\beta$ terms to see what the effects are.

In [189]:
import random
from PIL import Image
from IPython.display import HTML, display

alpha = 5
beta = 0
k = 100
width = 100
height = 100
X = random_init_matrix(width, height)

def ising_prior(alpha, beta, x, X, row, col, width, height):
    prob = -1 * alpha * x
    for n in get_neighbours(X, row, col, width, height):
        prob += -1 * beta * x * n
    return prob

def ising_normalized(alpha, beta, X, row, col, width, height):
    prob = math.exp(-1 * ising_prior(alpha, beta, X[row][col], X, row, col, width, height))
    normalize = math.exp(-1 * ising_prior(alpha, beta, 1, X, row, col, width, height))
    normalize += math.exp(-1 * ising_prior(alpha, beta, -1, X, row, col, width, height))
    return prob / normalize

def pixels_to_image(path, X, width, height):
    cmap = { -1 : (0, 0, 0), 1 : (255, 255, 255)}
    data = []
    for row in range(width):
        for col in range(height):
            data.append(cmap[X[row][col]])
    img = Image.new('RGB', (width, height), "blue")
    img.putdata(data)
    img.show()

for _ in range(k):
    for row in range(width):
        for col in range(height):
            if (row + col) % 2 == 0:
                continue
            if random.uniform(0, 1) < ising_normalized(alpha, beta, X, row, col, width, height):
                X[row][col] = 1
            else:
                X[row][col] = -1
    for row in range(100):
        for col in range(100):
            if (row + col) % 2 == 1:
                continue
            if random.uniform(0, 1) > ising_normalized(alpha, beta, X, row, col, width, height):
                X[row][col] = X[row][col] * -1
    pixels_to_image("test.png", X, width, height)

**(b)** Run your code on the images ${\tt noisy}$-${\tt message.png}$ and ${\tt noisy}$-${\tt yinyang.ong}$. After you read an image, you need to apply the following intensity transform to the pixels (to get black $= -1$ and white $= +1$): 

$x \ * \ 20 \ - \ 10$

Compute the posterior mean images for both examples (again, don't forget to burn-in). You can fix values for $\alpha$, $\beta$, and $\sigma^2$ (you will want to tune these manually to get something that works well). What values of $\alpha, \beta, \sigma^2$ did you use?

In [ ]:
# 1b

**(c)** Use your posterior samples to iteratively estimate $\sigma^2$ from the data. That is, assume the "clean" image that you sample is the true image, and use it to get an MLE of $\sigma^2$ (update this estimate each iteration). What final estimate do you get for $\sigma^2$?

In [2]:
# 1c

$2.$ Say you are given data $(X,Y)$, with $X \in \mathbb{R}^d$ and $Y \in \{0,1\}$. The goal is to train a classifier that will predict an unknown class label $\tilde{y}$ from a new data point $\tilde{x}$. Consider the following model:

$$Y \sim Ber\Big(\frac{1}{1+e^{-X^T\beta}}\Big),$$
$$\beta \sim N(0, \sigma^2I).$$

This is a Bayesian logistic regression model. Your goal is to derive and implement a Hamiltonian Monte Carlo sampler for doing Bayesian inference on $\beta$.

**(a)** Write down the formula for the unormalized posterior of $\beta|Y$, i.e.,

$$p(\beta|y;x,\sigma) \propto \prod_{i=1}^n p(y_i|\beta;x_i)p(\beta;\sigma)$$

**(b)** Show that this posterior is proportional to exp($-U(\beta))$), where

$$U(\beta) = \sum_{i=1}^n (1 - y_i)x_i^T \beta + log(1 + e^{-x_i^T \beta}) + \frac{1}{2\sigma^2}||\beta||^2$$

**(c)** Implement a Hamiltonian Monte Carlo Routine in Python for sampling from the posterior of $\beta$.

In [3]:
# 2c

**(d)** Use your code to analyze the ${\tt iris}$ data in Python, looking only at two species, *versicolor* and *virginica*. The species labels are your $Y$ data, and the four features, petal length and width, sepal length and width, are your $X$ data. Also, add a constant term, i.e., a columns of 1's to your $X$ matrix. Use the first 30 rows for each species as training data and leave out the last $20$ rows for each species as test data (for a total of $60$ training and $40$ testing). Generate samples of $\beta$ (don't forget to burn-in), and use these to get a prediction, $\tilde{y}$, of the class labels for the test data. Use your samples to get a Monte Carlo estimate of the posterior predictive probability $p(\tilde{y}|y)$ for each testing data point.

In [ ]:
# 2d

**(e)** Draw trace plots of your $\beta$ sequence and histograms (do 1D plots of each four vector components separately).

In [4]:
# 2e

**(f)** Compare this to the true class labels, $y$, and see how well you did by estimating the average error rate, $E[|y - \tilde{y}|]$ (a.k.a. the zero-one loss). What values of $\sigma$, $\epsilon$, and $L$ did you use?

In [ ]:
# 2f